In [1]:
from torch import from_numpy, Tensor
from torch import nn
from os.path import join
from os import getcwd

import pandas as pd
import numpy as np

In [2]:
pwd = getcwd()
datadir = join(pwd, 'data/')
file_train = join(datadir, 'train.csv')
file_test = join(datadir, 'test.csv')
file_challenge = join(datadir, 'challenge_data.csv')

In [3]:
df_train = pd.read_csv(file_train)
df_train['label'] = 'train'
df_test = pd.read_csv(file_test)
df_test['label'] = 'test'
df = pd.concat([df_train, df_test])

In [4]:
df.head()

,user_sequence,user_id,challenge_sequence,challenge,label
0,4576_1,4576,1,CI23714,train
1,4576_2,4576,2,CI23855,train
2,4576_3,4576,3,CI24917,train
3,4576_4,4576,4,CI23663,train
4,4576_5,4576,5,CI23933,train


In [5]:
df.tail()

,user_sequence,user_id,challenge_sequence,challenge,label
397315,113838_6,113838,6,CI23691,test
397316,113838_7,113838,7,CI24138,test
397317,113838_8,113838,8,CI23714,test
397318,113838_9,113838,9,CI24530,test
397319,113838_10,113838,10,CI23612,test


In [6]:
df.shape

(1301236, 5)

In [7]:
challenges = pd.read_csv(file_challenge)

In [8]:
challenges.head()

,challenge_ID,programming_language,challenge_series_ID,total_submissions,publish_date,author_ID,author_gender,author_org_ID,category_id
0,CI23478,2,SI2445,37.0,06-05-2006,AI563576,M,AOI100001,NaN
1,CI23479,2,SI2435,48.0,17-10-2002,AI563577,M,AOI100002,32.0
2,CI23480,1,SI2435,15.0,16-10-2002,AI563578,M,AOI100003,NaN
3,CI23481,1,SI2710,236.0,19-09-2003,AI563579,M,AOI100004,70.0
4,CI23482,2,SI2440,137.0,21-03-2002,AI563580,M,AOI100005,NaN


In [9]:
challenges = challenges.drop('challenge_ID', axis=1)
challenges = challenges.drop('author_gender', axis=1)
challenges = challenges.drop('author_org_ID', axis=1)
challenges['category_id'] = challenges['category_id'].fillna(0)
challenges['category_id'] = challenges['category_id'].astype(int)
# challenges['category_id'].value_counts()
challenges = challenges.drop('category_id', axis=1)

0      1841
29      412
39      208
69      170
122     170
       ... 
197       1
205       1
140       1
247       1
233       1
Name: category_id, Length: 195, dtype: int64